In [1]:
class Feature:
    '''
    This is a class for features from geojson.
    '''
    def __init__(self, features):
        # set the MBR for polygon and multipolygon:
        min_x = float('inf') # start with something much higher than expected min
        min_y = float('inf')
        max_x = float('-inf') # start with something much lower than expected max
        max_y = float('-inf')
        
        mbrs = []
        mbrs.append([])# list for main polygon
        mbrs.append([])# list for holes

        '''
        No holes:
            { "type": "Polygon",
                "coordinates": [
                                [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0] ]
                               ]
            }
            
        With holes:
            { "type": "Polygon",
              "coordinates": [
                              [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0] ],
                              [ [100.2, 0.2], [100.8, 0.2], [100.8, 0.8], [100.2, 0.8], [100.2, 0.2] ]
                             ]
            }
        '''
            
        if features['geometry']['type'] == 'Polygon':
            # for the polygon without holes:
            for item in features['geometry']['coordinates'][0]:   
                if item[0] < min_x:
                    min_x = item[0]
                if item[0] > max_x:
                    max_x = item[0]
                if item[1] < min_y:
                    min_y = item[1]
                if item[1] > max_y:
                    max_y = item[1]
            mbrs[0].append([min_x,max_x,min_y,max_y])
            # for the polygon with holes:
            if len(features['geometry']['coordinates'])>1:
                for polygons in features['geometry']['coordinates'][1:]: 
                    for item in polygons:
                        if item[0] < min_x:
                            min_x = item[0]
                        if item[0] > max_x:
                            max_x = item[0]
                        if item[1] < min_y:
                            min_y = item[1]
                        if item[1] > max_y:
                            max_y = item[1]
                    mbrs[1].append([min_x,max_x,min_y,max_y])
                '''
                The result is [[[x1,x2,y1,y2]],[[x3,x4,y3,y4],[x5,x6,y5,y6],[x7,x8,y7,y8]]]
                mbrs[0]--> for the main polygon
                mbrs[1]--> for all of holes in polygon
                '''
                '''
                ------------------------------------------------------------------------------------------
                '''
                
                '''
                { "type": "MultiPolygon",
                  "coordinates": [
                                    [
                                     [[102.0, 2.0], [103.0, 2.0], [103.0, 3.0], [102.0, 3.0], [102.0, 2.0]]
                                    ],# island without hole
                                    
                                    [
                                     [[100.0, 0.0], [101.0, 0.0], [101.0, 1.0], [100.0, 1.0], [100.0, 0.0]],
                                     [[100.2, 0.2], [100.8, 0.2], [100.8, 0.8], [100.2, 0.8], [100.2, 0.2]]
                                    ] # island with one hole
                                 ]
                }
                '''           

        elif features['geometry']['type'] == 'MultiPolygon':
            for island in features['geometry']['coordinates']:
                # each island could be island or island with holes
                if len(island)==1: # without hole
                    for polygon in island:
                        # the first polygon is main polygon, and else polygons should be the holes
                        for item in polygon:   
                            if item[0] < min_x:
                                min_x = item[0]
                            if item[0] > max_x:
                                max_x = item[0]
                            if item[1] < min_y:
                                min_y = item[1]
                            if item[1] > max_y:
                                max_y = item[1]
                        mbrs[0].append([min_x,max_x,min_y,max_y])

                if len(island)>1:# with holes
                    for item in island[0]:   
                        if item[0] < min_x:
                            min_x = item[0]
                        if item[0] > max_x:
                            max_x = item[0]
                        if item[1] < min_y:
                            min_y = item[1]
                        if item[1] > max_y:
                            max_y = item[1]
                    mbrs[0].append([min_x,max_x,min_y,max_y])  
                    for polygons in island[1:]:
                        for item in polygons:
                            if item[0] < min_x:
                                min_x = item[0]
                            if item[0] > max_x:
                                max_x = item[0]
                            if item[1] < min_y:
                                min_y = item[1]
                            if item[1] > max_y:
                                max_y = item[1]
                        mbrs[1].append([min_x,max_x,min_y,max_y])
                    '''
                    The result is [[[x1,x2,y1,y2],[x3,x4,y3,y4]],[[x5,x6,y5,y6],[x7,x8,y7,y8]]]
                    mbrs[0]--> for all the main polygons
                    mbrs[1]--> for all of holes in polygons
                    '''
        self.MBR = mbrs
        # read the three keys in dictionary for each feature
        self.geotype = features['type']
        self.geometry = features['geometry']
        self.properties = features['properties']
    
    # to judge a point(x,y) is in a rectangular(x1,x2,y1,y2) or not
    def inRect (self, x,y, rect):
        x1,x2,y1,y2 = rect
        if x<x2 and x>x1 and y<y2 and y>y1:
            return True
        else:
            return False
    # to judge a point(x,y) is in the MBR or not    
    def inMBR (self, x,y):
        if self.geometry['type'] == 'Polygon':
            mainP = self.inRect(x,y,self.MBR[0][0])
            if mainP == False:
                return False
            for i in range(len(self.MBR[1])):
                holeP = self.inRect(x,y,self.MBR[1][i])
                if holeP == True:
                    return False
            return True
        
        if self.geometry['type'] == 'MultiPolygon':
            for i in range(len(self.MBR[0])):
                mainP = self.inRect(x,y,self.MBR[0][i])
                if mainP == False:
                    return False
            for i in range(len(self.MBR[1])):
                holeP = self.inRect(x,y,self.MBR[1][i])
                if holeP == True:
                    return False
            return True

In [2]:
import json
f = "countries.geojson"
with open(f, 'r') as infile:
    g = json.load(infile)                        
    
# Convert all the GeoJSON features to Feature objects
features = []
for i in g["features"]:
    features.append(Feature(i)) 

In [3]:
from distance import to_float
from distance import get_attribute
from distance import read_csv

data = read_csv("data.csv")
lat = to_float(get_attribute(data,'latitude'))
lon = to_float(get_attribute(data,'longitude'))
ids = to_float(get_attribute(data,'longitude'))
points = list(zip(ids,lat,lon))

In [4]:
#import pyproj
#wgs84=pyproj.Proj("+init=EPSG:4326")
for country in features:
    AinR = []
    for i,airp in enumerate(points):
        #x,y = wgs84(airp[1],airp[2])
        if country.inMBR(airp[2],airp[1]) == True:
            # For each feature add a new attribute that lists the airports contained in its MBR.
            AinR.append(data[i])
    print(len(AinR))
    country.airList = AinR

1
56
0
1
5
0
0
0
0
0
0
0
0
0
1
0
69
0
1
39
3
7
0
7
1
1
8
0
1
15
0
0
36
1
1
0
1
48
10
0
57
0
0
0
0
24
0
0
17
1
0
1
0
0
0
0
1
1
0
0
32
1
1
2
1
0
91
0
0
0
0
1
0
48
0
0
0
0
0
0
0
1
10
0
7
0
14
1
0
0
0
1
0
15
2
0
0
0
0
0
0
26
0
1
0
0
1
0
1
28
1
24
0
6
1
17
0
1
0
0
0
0
0
0
0
1
2
0
40
3
5
25
2
0
0
4
0
1
8
2
2
40
1
5
0
0
0
1
2
18
3
0
2
21
1
0
0
2
1
0
0
35
1
16
1
0
0
1
1
0
38
1
0
0
0
0
0
0
0
0
0
0
54
0
0
0
22
1
0
3
33
0
4
5
0
0
0
19
14
0
5
0
0
0
1
0
0
5
18
1
9
1
28
12
11
0
1
1
1
13
0
9
2
0
0
0
0
0
1
1
0
0
1
0
15
0
0
18
7
1
0
0
0
0
0
2
1
1
1
1
2
0
0
25
20


In [5]:
# If I want to check the airport list in 
print("The country is %s."%(features[1].properties["ADMIN"]))
print(features[1].geometry['type'])
print(features[1].MBR)
features[1].airList

The country is Afghanistan.
Polygon
[[[60.48677779100012, 74.89230676300014, 29.38660532600005, 38.473673402000145]], []]


[['2047',
  'Wanganui',
  'Wanganui',
  'New Zealand',
  'WAG',
  'NZWU',
  '-39.962222',
  '175.025278',
  '27',
  '12',
  'Z'],
 ['2048',
  'Herat',
  'Herat',
  'Afghanistan',
  'HEA',
  'OAHR',
  '34.210017',
  '62.2283',
  '3206',
  '5',
  'U'],
 ['2049',
  'Jalalabad',
  'Jalalabad',
  'Afghanistan',
  'JAA',
  'OAJL',
  '34.399842',
  '70.498625',
  '1814',
  '5',
  'U'],
 ['2050',
  'Kabul Intl',
  'Kabul',
  'Afghanistan',
  'KBL',
  'OAKB',
  '34.565853',
  '69.212328',
  '5877',
  '5',
  'U'],
 ['2051',
  'Kandahar',
  'Kandahar',
  'Afghanistan',
  'KDH',
  'OAKN',
  '31.505756',
  '65.847822',
  '3337',
  '5',
  'U'],
 ['2052',
  'Maimana',
  'Maimama',
  'Afghanistan',
  'MMZ',
  'OAMN',
  '35.930789',
  '64.760917',
  '2743',
  '5',
  'U'],
 ['2053',
  'Mazar I Sharif',
  'Mazar-i-sharif',
  'Afghanistan',
  'MZR',
  'OAMS',
  '36.706914',
  '67.209678',
  '1284',
  '5',
  'U'],
 ['2054',
  'Shindand',
  'Shindand',
  'Afghanistan',
  '',
  'OASD',
  '33.391331',
  '62.2